In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import random
import logging as logger
import sqlite3

In [2]:
logger.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s',
                   level=logger.INFO,
                   datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
fldr = "C:/D/Projects/Vancouver"
outdb = sqlite3.connect(os.path.join(fldr, 'trip_table.sqlite'))

All geo layers for the STM were imported to a single Geopackage

In [4]:
nodelyr = sqlite3.connect(os.path.join(fldr, "Network/prepared_layer.gpkg"))
nds = pd.read_sql_query("SELECT fid, taz FROM nodes where pt_only=0", nodelyr)
tazs = {}
for taz in nds.taz.unique():
    tazs[taz] = nds[nds.taz==taz].fid.tolist()

Get the trips from the trip table

In [5]:
%%time
df = pd.read_sql_query("SELECT * FROM trips", outdb)
df.drop(['index'], axis=1, inplace=True)

Wall time: 6.24 s


In [6]:
def get_zone(taz, tazs):
    if taz not in tazs:
        return -1
    return random.choice(tazs[taz])

In [7]:
%%time
df = df.assign(from_node=-1)
df = df.assign(to_node=-1)

df['from_node'] = df.apply(lambda row: get_zone(row['rows'], tazs), axis=1)
df['to_node'] = df.apply(lambda row: get_zone(row['cols'], tazs), axis=1)

Wall time: 1min 15s


In [8]:
time_profile = pd.read_csv(os.path.join(fldr, 'nhts/dtc_dist.csv'))
distributions = {}

In [9]:
# AM distribution
am_dist = time_profile[450:510]
am_dist = am_dist.cumsum()[['WTTRDFIN']]
am_dist.WTTRDFIN /= am_dist.WTTRDFIN.max()
distributions['Am'] = am_dist

In [10]:
# MD distribution
md_dist = time_profile[720:780]
md_dist = md_dist.cumsum()[['WTTRDFIN']]
md_dist.WTTRDFIN /= md_dist.WTTRDFIN.max()
distributions['Md'] = md_dist

In [11]:
# PM distribution
pm_dist = time_profile[990:1050]
pm_dist = pm_dist.cumsum()[['WTTRDFIN']]
pm_dist.WTTRDFIN /= pm_dist.WTTRDFIN.max()
distributions['Pm'] = pm_dist

In [12]:
def get_exact_time(mat_class, distributions):
    period = mat_class[-2:]
    dist = distributions[period]
    rdn = np.random.rand(1)[0]
    return dist.index[np.argmax(dist.WTTRDFIN.values>rdn)] + np.random.rand(1)[0]

In [13]:
%%time
df = df.assign(departure_minute=-1)
df['departure_minute'] = df.apply(lambda row: get_exact_time(row['class'], distributions), axis=1)

Wall time: 1min 41s


In [14]:
df.to_sql('distributed_trips', outdb, if_exists='replace')